In [1]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute,IBMQ
from qiskit.tools.monitor import job_monitor
from qiskit.circuit.library import QFT
from qiskit.circuit.library.standard_gates import PhaseGate
from qiskit import Aer
import numpy as np

backend = Aer.get_backend('statevector_simulator')
   

In [2]:
def d2b(n):  
    return bin(n).replace("0b", "") 

In [3]:

def multiplier(x,y):
    x_b=d2b(x)
    y_b=d2b(y)
    n=max(len(x_b),len(y_b))
    
    xr = QuantumRegister(n, name="xr")
    yr = QuantumRegister(n, name="yr")
    zr = QuantumRegister(2*n, name="zr")
    c = ClassicalRegister(2*n,'c')

    qc = QuantumCircuit()
    qc.add_register(xr,yr,zr,c)

    #Encoding
    a=0
    for i in "".join(reversed(x_b)):
        if i=='1':
            qc.x(xr[a])
        a+=1
    a=0
    for i in "".join(reversed(y_b)):
        if i=='1':
            qc.x(yr[a])
        a+=1

    # build multiplication circuit

    qc.append(QFT(2*n, do_swaps=False).to_gate(), zr[:])

    for j in range(1, n + 1):
        for i in range(1, n + 1):
            for k in range(1, 2*n + 1):
                lam = (2 * np.pi) / (2 ** (i + j + k - 2 * n))
                qc.append(PhaseGate(lam).control(2),[xr[n - j], yr[n - i], zr[k - 1]],)

    qc.append(QFT(2*n, do_swaps=False).inverse().to_gate(), zr[:])

    for i in range(2*n):
        qc.measure(zr[int(i)],c[i])
 

    print(qc)

    job = execute(qc, backend, shots=2000)
    result = job.result()
    counts = result.get_counts()
    ans=list(counts.keys())
    z=0
    a=0
    for i in "".join(reversed(ans[0])):
        z+=int(i)*(2**a)
        a+=1
    
    return z

In [4]:
multiplier(10,2)

                                                                        »
xr_0: ──────────────────────────────────────────────────────────────────»
       ┌───┐                                                            »
xr_1: ─┤ X ├────────────────────────────────────────────────────────────»
       └───┘                                                            »
xr_2: ──────────────────────────────────────────────────────────────────»
       ┌───┐                                                            »
xr_3: ─┤ X ├───■────────■────────■────────■───────■───────■───────■─────»
       └───┘   │        │        │        │       │       │       │     »
yr_0: ─────────┼────────┼────────┼────────┼───────┼───────┼───────┼─────»
       ┌───┐   │        │        │        │       │       │       │     »
yr_1: ─┤ X ├───┼────────┼────────┼────────┼───────┼───────┼───────┼─────»
       └───┘   │        │        │        │       │       │       │     »
yr_2: ─────────┼────────┼────────┼────

20